# Text Generation

在Chatbot 中，需要自动生成text。

通常的流程是：
1. 给定一个set of parameters，
2. 生成一个set of text
3. 选出得分最高的candidate

下面我们看，如何使用LSTM 生成text。

通常，基于概率模型的文本生成方法，例如马尔可夫方法，是计算一个条件概率分布，即基于前面几个单词(n-gram)， 预测下一个单词的概率。RNN 和LSTM 的方法类似，区别在于：
- RNN encode information -- feature extraction
- LSTM: memory state has greater context -- better performance

下面我们介绍，如何使用LSTM 自动生成text.

如果要做prediction，我们需要修改network 的结构。这不是一个sentiment analysis 的分类问题了。这个word embedding 类似，是一个self-supervised learning. 

<img src="img/next_word_prediction.png" alt="drawing" width="600"/>

这里，我们不使用之前的IMDB 数据库，原因是：
- 数据库小
- 异质化比较严重。异质化的意思是，review 是不同人写的，大家有不同的书写风格。

所以我们下面通过学习莎士比亚的文章，来生成莎士比亚风格的文本(singular style)。

In [1]:
from nltk.corpus import gutenberg

In [2]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


获取所有莎士比亚的作品，并把他们拼接成一个large string。

In [3]:
text = ''
for txt in gutenberg.fileids():
    if 'shakespeare' in txt:
        text += gutenberg.raw(txt).lower()

print('corpus length:', len(text))

corpus length: 375542


In [7]:
print(text[:500])

[the tragedie of julius caesar by william shakespeare 1599]


actus primus. scoena prima.

enter flauius, murellus, and certaine commoners ouer the stage.

  flauius. hence: home you idle creatures, get you home:
is this a holiday? what, know you not
(being mechanicall) you ought not walke
vpon a labouring day, without the signe
of your profession? speake, what trade art thou?
  car. why sir, a carpenter

   mur. where is thy leather apron, and thy rule?
what dost thou with thy best apparrell on


下面，我们统计所有出现过的characters，类似于构建字典。

In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 50


In [5]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

下面，我们构建一个training set. 构建方法：对于刚才构建的text string，我们选取大小为40的滑动窗口，step = 3，构建训练集。

In [8]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 125168


In [12]:
sentences[0], next_chars[0]

('[the tragedie of julius caesar by willia', 'm')

我们构建了125,168 个这样的sequences 作为训练集。下面，我们对每个sequence 做one-hot 编码。

In [14]:
import numpy as np

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


下面，我们构建model.

- 我们不需要每个步骤的输出，只需要最后一个输出，所以不需要`return_sequences=True` 参数
- 因为这个问题更复杂，所以我们使用了更大的network，LSTM 有128 个neurons
- 使用RMSprop 作为优化器
- loss function
- no dropout: 我们需要生成text 和莎士比亚越像越好，所以追求过拟合。所以这和传统的方法追求泛化是不同的。


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

print(model.summary())